# 🚢 Boat Type Classification - Optimized Version

This notebook trains a boat classification model using **MobileNetV2** with proper regularization to prevent overfitting.

## What this notebook does:
1. Splits dataset into train/validation/test (70%/15%/15%)
2. Applies data augmentation to increase training data variety
3. Trains MobileNetV2 with dropout and regularization
4. Evaluates performance and saves the best model

**Expected accuracy: 75-85%** (depends on dataset size and quality)

In [ ]:
# Step 1: Import Required Libraries
# =====================================
# These libraries help us build and train the deep learning model

import numpy as np                                      # For numerical operations
import tensorflow as tf                                # Deep learning framework
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # For loading and augmenting images
from tensorflow.keras.models import Sequential         # For building sequential model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization  # Model layers
from tensorflow.keras.applications import MobileNetV2  # Pre-trained model
from tensorflow.keras.optimizers import Adam           # Optimizer for training
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  # Training callbacks
from tensorflow.keras.regularizers import l2           # L2 regularization to prevent overfitting
import matplotlib.pyplot as plt                        # For plotting graphs
import seaborn as sns                                  # For better visualizations
from sklearn.metrics import classification_report, confusion_matrix  # Performance metrics
import os
import shutil
import random

print("✅ All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# Step 2: Split Dataset into Train/Validation/Test Sets
# ======================================================
# This ensures we can properly evaluate our model on unseen data

base_dir = '../boat_type_classification_dataset'  # Original dataset location
output_dir = './data'                             # Where to save split data

# Create directories for each split
train_dir = os.path.join(output_dir, 'train')
validation_dir = os.path.join(output_dir, 'validation')
test_dir = os.path.join(output_dir, 'test')

# Remove existing data directory if it exists (to start fresh)
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print("🗑️  Removed existing data directory")

# Create new directories
os.makedirs(train_dir)
os.makedirs(validation_dir)
os.makedirs(test_dir)
print("📁 Created train/validation/test directories")

# Get list of boat type classes (folder names)
classes = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
print(f"\n📊 Found {len(classes)} boat types: {classes}")

# Split ratio (industry standard)
train_ratio = 0.7       # 70% for training the model
validation_ratio = 0.15  # 15% for tuning hyperparameters during training
test_ratio = 0.15        # 15% for final evaluation (unseen data)

print(f"\n📈 Split ratios: Train={train_ratio*100}%, Val={validation_ratio*100}%, Test={test_ratio*100}%")

# Process each boat class
for cls in classes:
    # Create subdirectories for this class in train/val/test
    os.makedirs(os.path.join(train_dir, cls))
    os.makedirs(os.path.join(validation_dir, cls))
    os.makedirs(os.path.join(test_dir, cls))

    # Get all image files for this class
    src_dir = os.path.join(base_dir, cls)
    all_files = [f for f in os.listdir(src_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Shuffle to ensure random distribution
    random.shuffle(all_files)

    # Calculate split indices
    train_split = int(len(all_files) * train_ratio)
    validation_split = int(len(all_files) * (train_ratio + validation_ratio))

    # Split files into three sets
    train_files = all_files[:train_split]
    validation_files = all_files[train_split:validation_split]
    test_files = all_files[validation_split:]

    # Copy files to their respective directories
    for f in train_files:
        shutil.copy(os.path.join(src_dir, f), os.path.join(train_dir, cls, f))
    for f in validation_files:
        shutil.copy(os.path.join(src_dir, f), os.path.join(validation_dir, cls, f))
    for f in test_files:
        shutil.copy(os.path.join(src_dir, f), os.path.join(test_dir, cls, f))
    
    print(f"   {cls:<18} Total: {len(all_files):>3} → Train: {len(train_files):>3}, Val: {len(validation_files):>2}, Test: {len(test_files):>2}")

print("\n✅ Dataset split completed successfully!")

In [ ]:
# Step 3: Setup Data Generators with Augmentation
# ================================================
# Data augmentation creates variations of training images to prevent overfitting

img_size = (224, 224)  # MobileNetV2 expects 224x224 images
batch_size = 32        # Number of images processed at once (standard size)

# Training data generator with augmentation
# ==========================================
# Augmentation helps the model learn from variations of the same image
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize pixel values from [0-255] to [0-1]
    rotation_range=20,           # Randomly rotate images by up to 20 degrees
    width_shift_range=0.15,      # Randomly shift images horizontally by 15%
    height_shift_range=0.15,     # Randomly shift images vertically by 15%
    shear_range=0.15,            # Apply random shearing transformations
    zoom_range=0.15,             # Randomly zoom in/out by 15%
    horizontal_flip=True,        # Randomly flip images horizontally (boats can face either direction)
    fill_mode='nearest'          # Fill empty pixels with nearest neighbor
)

# Validation and test data generators (NO augmentation)
# ======================================================
# We only normalize validation/test data, no augmentation needed
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,        # Resize all images to 224x224
    batch_size=batch_size,       # Process 32 images at a time
    class_mode='categorical',    # Multi-class classification (9 boat types)
    shuffle=True                 # Shuffle training data for better learning
)

val_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False                # Don't shuffle validation data (not needed)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False                # Don't shuffle test data
)

# Display class distribution
print("\n📊 Dataset Summary:")
print(f"   Training samples: {train_generator.samples}")
print(f"   Validation samples: {val_generator.samples}")
print(f"   Test samples: {test_generator.samples}")
print(f"   Number of classes: {train_generator.num_classes}")
print(f"   Class names: {list(train_generator.class_indices.keys())}")
print("\n✅ Data generators created successfully!")

In [ ]:
# Step 4: Build MobileNetV2 Model with Regularization
# ====================================================
# We use transfer learning: take a pre-trained model and adapt it for our task

print("🔨 Building MobileNetV2 model...\n")

# Load pre-trained MobileNetV2 (trained on ImageNet with 1.4M images)
base_model = MobileNetV2(
    weights='imagenet',          # Use weights pre-trained on ImageNet dataset
    include_top=False,           # Remove the final classification layer (we'll add our own)
    input_shape=(224, 224, 3)    # Input shape: 224x224 RGB images
)

# Freeze base model weights initially (we don't want to destroy pre-trained features)
base_model.trainable = False
print("   ✓ Loaded pre-trained MobileNetV2")
print("   ✓ Froze base model weights (transfer learning)")

# Build complete model by adding custom classification layers on top
model = Sequential([
    # Pre-trained MobileNetV2 base (feature extractor)
    base_model,
    
    # Pooling layer: reduces each feature map to a single number
    GlobalAveragePooling2D(),
    
    # First dense layer: 256 neurons with L2 regularization to prevent overfitting
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),        # Normalize activations for stable training
    Dropout(0.5),                # Drop 50% of neurons randomly to prevent overfitting
    
    # Second dense layer: 128 neurons
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.3),                # Drop 30% of neurons
    
    # Output layer: 9 neurons (one for each boat type)
    Dense(train_generator.num_classes, activation='softmax')  # Softmax converts to probabilities
], name='BoatClassifier_MobileNetV2')

# Compile model: specify optimizer, loss function, and metrics
model.compile(
    optimizer=Adam(learning_rate=0.0001),    # Adam optimizer with low learning rate
    loss='categorical_crossentropy',         # Loss function for multi-class classification
    metrics=['accuracy']                     # Track accuracy during training
)

# Display model architecture
print("\n📋 Model Architecture:")
model.summary()

# Count parameters
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
total_params = model.count_params()
frozen_params = total_params - trainable_params

print(f"\n📊 Parameter Summary:")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Frozen parameters: {frozen_params:,}")
print("\n✅ Model built successfully!")

In [ ]:
# Step 5: Setup Training Callbacks
# =================================
# Callbacks help prevent overfitting and optimize training

# Early Stopping: stops training if validation loss doesn't improve
# ===================================================================
# Monitors validation loss and stops if no improvement for 'patience' epochs
early_stopping = EarlyStopping(
    monitor='val_loss',          # Watch validation loss
    patience=7,                  # Wait 7 epochs for improvement
    restore_best_weights=True,   # Restore weights from best epoch
    verbose=1                    # Print when stopping
)

# Learning Rate Reduction: reduces learning rate when validation loss plateaus
# ===========================================================================
# If validation loss stops improving, reduce learning rate to fine-tune
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',          # Watch validation loss
    factor=0.5,                  # Reduce learning rate by 50%
    patience=3,                  # Wait 3 epochs before reducing
    min_lr=1e-7,                 # Don't go below this learning rate
    verbose=1                    # Print when reducing
)

print("✅ Training callbacks configured:")
print("   ✓ Early Stopping (patience=7) - prevents overfitting")
print("   ✓ Learning Rate Reduction (patience=3) - fine-tunes learning")

In [ ]:
# Step 6: Train the Model
# =======================
# This is where the model learns to classify boats

print("\n🚀 Starting training...")
print("=" * 70)
print("📊 Training Configuration:")
print(f"   Max epochs: 50")
print(f"   Batch size: {batch_size}")
print(f"   Learning rate: 0.0001")
print(f"   Training samples: {train_generator.samples}")
print(f"   Validation samples: {val_generator.samples}")
print("=" * 70)
print("\n⏱️  This will take approximately 20-30 minutes...\n")

# Train the model
history = model.fit(
    train_generator,                         # Training data
    validation_data=val_generator,           # Validation data for monitoring
    epochs=50,                               # Maximum number of epochs
    callbacks=[early_stopping, reduce_lr],   # Use callbacks to prevent overfitting
    verbose=1                                # Show progress bar
)

print("\n✅ Training completed!")
print(f"   Total epochs trained: {len(history.history['accuracy'])}")
print(f"   Final training accuracy: {history.history['accuracy'][-1]*100:.2f}%")
print(f"   Final validation accuracy: {history.history['val_accuracy'][-1]*100:.2f}%")

In [ ]:
# Step 7: Visualize Training History
# ===================================
# Plot accuracy and loss curves to diagnose training

plt.figure(figsize=(14, 5))

# Plot 1: Accuracy over epochs
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
plt.title('Model Accuracy Over Time', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend(loc='lower right')
plt.grid(alpha=0.3)

# Plot 2: Loss over epochs
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
plt.title('Model Loss Over Time', fontsize=14, fontweight='bold')
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend(loc='upper right')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Interpretation guide
print("\n📊 How to Read These Plots:")
print("\n✅ Good Signs (Model is learning well):")
print("   • Training and validation curves are close together")
print("   • Both accuracies increase over time")
print("   • Both losses decrease over time")
print("\n⚠️  Warning Signs (Overfitting):")
print("   • Large gap between training and validation accuracy")
print("   • Training loss keeps decreasing but validation loss increases")
print("   • If you see this, the model memorized training data instead of learning patterns")

In [ ]:
# Step 8: Evaluate on Test Set
# =============================
# Test on completely unseen data to get true performance

print("📊 Evaluating model on test set...\n")

# Evaluate model
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

print("\n" + "=" * 70)
print("🎯 FINAL TEST RESULTS")
print("=" * 70)
print(f"   Test Accuracy: {test_acc*100:.2f}%")
print(f"   Test Loss: {test_loss:.4f}")
print("=" * 70)

# Interpretation
if test_acc >= 0.80:
    print("\n✅ EXCELLENT! Accuracy above 80% - Model is ready for deployment!")
elif test_acc >= 0.70:
    print("\n✅ GOOD! Accuracy above 70% - Model works well, can be improved with more data")
elif test_acc >= 0.60:
    print("\n⚠️  FAIR! Accuracy above 60% - Model needs more training data or tuning")
else:
    print("\n❌ POOR! Accuracy below 60% - Consider collecting more data or trying different architecture")

In [ ]:
# Step 9: Detailed Performance Analysis
# ======================================
# Generate predictions and analyze per-class performance

print("🔍 Generating predictions for detailed analysis...\n")

# Get predictions for all test images
y_pred = np.argmax(model.predict(test_generator), axis=1)
y_true = test_generator.classes
class_labels = list(train_generator.class_indices.keys())

# Classification Report: shows precision, recall, F1-score for each class
print("📋 Classification Report:")
print("=" * 70)
print(classification_report(y_true, y_pred, target_names=class_labels))

print("\n💡 Understanding the Metrics:")
print("   • Precision: Of all predictions for this class, how many were correct?")
print("   • Recall: Of all actual images of this class, how many did we find?")
print("   • F1-Score: Harmonic mean of precision and recall (overall quality)")
print("   • Support: Number of test images for this class")

In [ ]:
# Step 10: Confusion Matrix Visualization
# ========================================
# See which classes the model confuses with each other

print("📊 Creating confusion matrix...\n")

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Create visualization
plt.figure(figsize=(12, 10))
sns.heatmap(
    conf_matrix, 
    annot=True,              # Show numbers in cells
    fmt='d',                 # Format as integers
    cmap='Blues',            # Color scheme
    xticklabels=class_labels,
    yticklabels=class_labels,
    cbar_kws={'label': 'Number of Images'}
)
plt.xlabel('Predicted Class', fontsize=12, fontweight='bold')
plt.ylabel('Actual Class', fontsize=12, fontweight='bold')
plt.title(f'Confusion Matrix - Test Accuracy: {test_acc*100:.2f}%', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n💡 How to Read the Confusion Matrix:")
print("   • Diagonal (top-left to bottom-right): Correct predictions")
print("   • Off-diagonal: Misclassifications")
print("   • Darker colors = more images in that cell")
print("   • Perfect model would have ALL numbers on the diagonal")

In [ ]:
# Step 11: Save the Trained Model
# ================================
# Save model to use in the web application

model_filename = 'boat_classifier_mobilenet.h5'
model.save(model_filename)

print("\n" + "=" * 70)
print("💾 MODEL SAVED SUCCESSFULLY")
print("=" * 70)
print(f"   Filename: {model_filename}")
print(f"   Location: {os.path.abspath(model_filename)}")
print(f"   File size: {os.path.getsize(model_filename) / (1024*1024):.2f} MB")
print("\n📝 Next Steps:")
print("   1. Move the model to backend folder:")
print(f"      Move-Item -Path '{model_filename}' -Destination 'backend/{model_filename}' -Force")
print("   2. Start the backend server:")
print("      cd backend; python app.py")
print("   3. Open frontend/index.html in your browser")
print("   4. Test with boat images!")
print("=" * 70)

## 🎉 Training Complete!

### Summary of What We Did:

1. ✅ Split dataset (70% train, 15% validation, 15% test)
2. ✅ Applied data augmentation to prevent overfitting
3. ✅ Built MobileNetV2 model with regularization
4. ✅ Trained with early stopping and learning rate reduction
5. ✅ Evaluated on unseen test data
6. ✅ Analyzed per-class performance
7. ✅ Saved model for deployment

### How to Improve Accuracy Further:

1. **Collect More Data** (Most Important!)
   - Classes with few images (<20) will perform poorly
   - Aim for 100+ images per class for best results

2. **Try Fine-Tuning**
   - Unfreeze top layers of MobileNetV2 and train with lower learning rate

3. **Try Different Architectures**
   - EfficientNetB3: Better accuracy but slower
   - ResNet50: Good for complex features
   - InceptionV3: Good for varied image sizes

### Expected Performance:
- **With balanced data (100+ images/class)**: 85-90% accuracy
- **With imbalanced data**: 70-80% accuracy
- **Production systems typically achieve**: 75-85% accuracy